In [88]:
import torch
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
print("Device: {}".format(device))

Device: cpu


In [89]:
import numpy as np
import os
from get_data import get_data
from imblearn.over_sampling import SMOTE 
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from numpy.random import default_rng
rng = default_rng(seed = 31)
import pickle
import pandas as pd
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score
import geopandas as gpd
import shapely
import libpysal
import matplotlib.pyplot as plt
from copy import deepcopy
from sklearn.cluster import KMeans
from collections import Counter

In [90]:
from torch import nn, optim
import scipy.ndimage as ndimage

In [91]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [110]:
def load_data(year):
    os.getcwd()
    df = pd.DataFrame()
    path = "../Data/filled/" + str(year) + "/"
    for filename in os.listdir(path):
        df1 = pd.read_csv(path + filename)
        print(filename)
        if df1.geometry.isna().any():
            print(filename)
        df = pd.concat([df, df1])
        
    df = gpd.GeoDataFrame(df)
    df.geometry = df.geometry.apply(shapely.wkt.loads)
#     df.plot(figsize = (40,40), column = "y")
#     plt.show()
    df = df.reset_index()
    df = df.drop(["Unnamed: 0", "index"], axis = 1)
    return df 

In [111]:
# df15 = load_data(2015)
# df16 = load_data(2016)
df17 = load_data(2017)
df18 = load_data(2018)
df19 = load_data(2019)

wt = libpysal.weights.DistanceBand.from_dataframe(df17, threshold=150, binary = True, silence_warnings = True)
wv = libpysal.weights.DistanceBand.from_dataframe(df18, threshold=150, binary = True, silence_warnings = True)
wtest = libpysal.weights.DistanceBand.from_dataframe(df19, threshold=150, binary = True, silence_warnings = True)

's-Gravenhage17.csv_filled_.csv
's-Gravenhage18.csv_filled_.csv
's-Gravenhage19.csv_filled_.csv


In [112]:
cluster_train = df17[df17.columns[:-5]]
cluster_val = df18[df18.columns[:-5]]
cluster_test = df19[df19.columns[:-5]]
cluster_train = ss.fit_transform(cluster_train)
cluster_val = ss.transform(cluster_val)
cluster_test = ss.transform(cluster_test)


c=3
cluster_model = KMeans(n_clusters = c, random_state = 1)
train_l = pd.DataFrame(cluster_model.fit_predict(cluster_train))
val_l = pd.DataFrame(cluster_model.predict(cluster_val))
test_l= pd.DataFrame(cluster_model.predict(cluster_test))

In [113]:
Counter(train_l[0])

Counter({2: 1242, 0: 4894, 1: 2882})

In [114]:
Counter(val_l[0])

Counter({2: 1282, 0: 4897, 1: 2932})

In [115]:
Counter(test_l[0])

Counter({2: 1239, 0: 5011, 1: 2932})

In [116]:
def create_CNN_samples(grid, block, dims = 39):
    
    nonzero = np.transpose(grid[:,:,-2].nonzero()) # Get indices of nonzero componetns
    size = nonzero.shape[0]
    width = block * 2 + 1 # calculate widht and height. Needed later on
    
    X = np.zeros((size, width, width, dims))
    Y = np.zeros(size)
    ID = np.zeros(size)
    Y_1 = np.zeros(size)
    
    for idx, i in enumerate(nonzero):
        x, ID[idx], Y[idx], Y_1[idx] = get_neighbor_grid(grid, i, block)
        X[idx] = x.reshape(width,width, dims)
        
    X = np.moveaxis(X, -1, 1) # order the indices correctly to make sure it works in CNN
    X = torch.from_numpy(X).float()
    Y = torch.from_numpy(Y).float()
    
    return X,ID,Y, Y_1

In [117]:
def get_neighbor_grid(full, hw, block = 1):
    
    # get the nonzero (built) blocks by checking if they have a ID

    h = hw[0]
    w = hw[1]
    
    y = full[h,w,-1]
    ID = full[h,w,-2]
    Y_1_train = full[h,w,-3]
    
    hu = h - block
    hd = h + block
    hshort, hextra, wshort, wextra = 0,0,0,0
    if hu < 0:
        hshort = 0 - hu
        hu = 0
    if hd >= full.shape[0]:
        hextra = (hd - full.shape[0]) + 1
        hd = full.shape[0]

    wr = w + block
    wl = w - block

    if wr >= full.shape[1]:
        wextra = (wr - full.shape[1]) + 1
        wr = full.shape[1]
    if wl < 0:
        wshort = 0 - wl
        wl = 0

    nb = full[hu : hd + 1, wl : wr + 1, :]
    nb = np.pad(nb, ((hshort, hextra), (wshort, wextra), (0,0)), mode = "constant", constant_values = 0)
    return nb[:,:,:-3], ID, y, Y_1_train


In [118]:
##  trainingset
X_train = []
Y_train = []
ID_train = []
Y_1_train = []
for filename in os.listdir("../Data/filled/grids/2017/"):
    print(filename)
    n = np.load("../Data/filled/grids/2017/" + filename)
    X, ID, Y, Y_1 = create_CNN_samples(n, 10)
    X_train.append(X)
    Y_train.append(Y)
    ID_train.append(ID)
    Y_1_train.append(Y_1)
    
block_size = X.shape[-1]
    
Y_train = np.concatenate(Y_train)
ID_train = np.concatenate(ID_train)
X_train = np.concatenate(X_train)

# create Y_1_train
Y_1_train = np.concatenate(Y_1_train)
neighbors = [wt.neighbors[x] for x in wt.neighbors]
Y_1_train = np.array([Y_1_train[x].sum() for x in neighbors])
Y_1_train[np.where(Y_1_train == 0)[0]] = 0.5

#reshape to rescale
X_train = np.moveaxis(X_train, 1, -1)
X_train = X_train.reshape(-1, 39)
X_train = ss.fit_transform(X_train) #rescale
# reshape to oversample, append y_1 to resample
X_train = X_train.reshape(-1, block_size, block_size, 39)
X_train = X_train.reshape(X_train.shape[0], -1)



## validation set
X_val = []
Y_val = []
ID_val = []
Y_1_val = []

for filename in os.listdir("../Data/filled/grids/2018/"):
    print(filename)
    n = np.load("../Data/filled/grids/2018/" + filename)
    X, ID, Y, Y_1 = create_CNN_samples(n, 10)
    X_val.append(X)
    Y_val.append(Y)
    ID_val.append(ID)
    Y_1_val.append(Y_1)
    
# create and transform X_val
X_val = np.concatenate(X_val)
X_val = np.moveaxis(X_val, 1, -1)
X_val = X_val.reshape(-1, 39)
X_val = ss.transform(X_val)
X_val = X_val.reshape(-1, block_size, block_size, 39)
X_val = np.moveaxis(X_val, -1, 1)


# create Y_1_val
Y_1_val = np.concatenate(Y_1_val)
neighbors = [wv.neighbors[x] for x in wv.neighbors]
Y_1_val = np.array([Y_1_val[x].sum() for x in neighbors])
Y_1_val[np.where(Y_1_val == 0)[0]] = 0.5

# create Y-val and ID_val
Y_val = np.concatenate(Y_val)
ID_val = np.concatenate(ID_val)
X_val = torch.tensor(X_val).float().to(device)
Y_1_val = torch.tensor(Y_1_val).float().to(device)

## test set
X_test = []
Y_test = []
ID_test = []
Y_1_test = []

for filename in os.listdir("../Data/filled/grids/2019/"):
    print(filename)
    n = np.load("../Data/filled/grids/2019/" + filename)
    X, ID, Y, Y_1 = create_CNN_samples(n, 10)
    X_test.append(X)
    Y_test.append(Y)
    ID_test.append(ID)
    Y_1_test.append(Y_1)
    
# create and transform X_val
X_test = np.concatenate(X_test)
X_test = np.moveaxis(X_test, 1, -1)
X_test = X_test.reshape(-1, 39)
X_test = ss.transform(X_test)
X_test = X_test.reshape(-1, block_size, block_size, 39)
X_test = np.moveaxis(X_test, -1, 1)


# create Y_1_val
Y_1_test = np.concatenate(Y_1_test)
neighbors = [wtest.neighbors[x] for x in wtest.neighbors]
Y_1_test = np.array([Y_1_test[x].sum() for x in neighbors])
Y_1_test[np.where(Y_1_test == 0)[0]] = 0.5

# create Y-val and ID_val
Y_test = np.concatenate(Y_test)
ID_test = np.concatenate(ID_test)
X_test = torch.tensor(X_test).float().to(device)
Y_1_test = torch.tensor(Y_1_test).float().to(device)

's-Gravenhage17.npy
's-Gravenhage18.npy
's-Gravenhage19.npy


In [119]:
def get_batch(X, Y, Y1, batch_size = 32):
    
    idxs = rng.integers(X.shape[0], size = batch_size)
    
    X = X[idxs]
    Y = Y[idxs]
    Y1 = Y1[idxs]

    return X, Y, Y1


In [120]:
class Model2(nn.Module): # best auc = 0.58
    def __init__(self):
        super(Model2, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 15, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(), 
            nn.Dropout2d(0.3),
            nn.Conv2d(in_channels = 15, out_channels = 20, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(in_channels = 20, out_channels = 5, kernel_size = (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(5, 1)
            )
        
        self.final = nn.Sigmoid()
        
    def forward(self, x, y1):
#         for i in self.net:
            
        out = self.net(x)
        out = out.squeeze() * y1
        return out

In [121]:
class Model1(nn.Module): # best auc = 0.58
    def __init__(self):
        super(Model1, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 15, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(), 
            nn.Dropout2d(0.3),
            nn.Conv2d(in_channels = 15, out_channels = 20, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Flatten(),
            nn.Linear(180 , 150),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(150,1)
            )
        
        self.final = nn.Sigmoid()
        
    def forward(self, x, y1):
#         for i in self.net:
            
        out = self.net(x)
        out = out.squeeze() * y1
        return out

In [122]:

class Model3(nn.Module): # best auc = 0.58
    def __init__(self):
        super(Model3, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 5, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(), 
            nn.Dropout2d(0.3),
            nn.Conv2d(in_channels = 5, out_channels = 5, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Flatten(),
            nn.Linear(45, 1)
            )
        
        self.final = nn.Sigmoid()
        
    def forward(self, x, y1):
#         for i in self.net:
            
        out = self.net(x)
        out = out.squeeze() * y1
        return out

In [123]:
class Model(nn.Module): # best auc = 0.58
    def __init__(self):
        super(Model, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 5, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(), 
            nn.Dropout2d(0.3),
            nn.Conv2d(in_channels = 5, out_channels = 15, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Flatten(),
            nn.Linear(135, 150),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(150,1)
            )
        
        self.final = nn.Sigmoid()
        
    def forward(self, x, y1):
#         for i in self.net:
            
        out = self.net(x)
        out = out.squeeze() * y1
        return out

In [124]:
model = Model().to(device)
model1 = Model1().to(device)
model2 = Model2().to(device)
model3 = Model3().to(device)

models = ["0","1", "2", "3"]

In [125]:
print(X_train.shape, X_val.shape) # wat is de volgorde van X_train en X_val?

(9018, 17199) torch.Size([9111, 39, 21, 21])


In [126]:
def train_model(models, X_train, Y_train, Y1_train, train_l, X_val, Y_val, Y1_val, val_l, num_epochs, lr, batch_per_e = 100):

    tot_auc = 0
    tot_model = np.NaN
    tot_param = 0
    
    for c in [3]:

        print("{}: train: {}, val: {}".format(c,Counter(train_l[0]), Counter(val_l[0])))
        
        xtrain = np.append(X_train, Y1_train.reshape(-1,1), axis = 1) # append Y1 for resampling 
        
        
        auc_clust_hist = {}
        f1_clust_hist = {}
        clust_models = {}
        param_clust_hist = {} 

        for clust in train_l[0].unique():
        
            best_auc = 0
            best_f1 = 0
            best_model = np.NaN

            idxs = train_l[train_l[0]== clust].index
            xt = xtrain[idxs]
            yt = Y_train[idxs]
            
            if np.sum(yt==1) >10:
                oversample = SMOTE()
                xt, yt = oversample.fit_resample(xt, yt)
            
            xt[xt[:,-1]>0.5][:,-1] =  np.ceil(xt[xt[:,-1] > 0.5][:,-1]) # make y1 great again
            y1t = xt[:,-1] # get y1_train
            xt = np.delete(xt, 39, 1) 

            # reshape to grid
            xt = xt.reshape(-1, block_size, block_size, 39)

            # move axis for pytorch
            xt = np.moveaxis(xt, -1, 1) 
            
            xt = torch.tensor(xt).float().to(device)
            yt = torch.tensor(yt).float().to(device)
            y1t = torch.tensor(y1t).float().to(device)
     

            # validation set for the clusters
            idxs = val_l[val_l[0] == clust].index
            xv = X_val[idxs]
            yv = Y_val[idxs]
            y1v = Y1_val[idxs]

            yv = torch.tensor(yv).float().to(device)
            yn = yv.cpu().detach().numpy()
            

            for lr in [0.00000051, 0.0000051]:
                for size in models:
                    if size == "0":
                        model = Model()
                    elif size == "1":
                        model = Model1()
                    elif size == "2":
                        model = Model2()
                    else:
                        size = Model3()
                

                    optimizer = optim.Adam(model.parameters(), lr = lr)
                    SigBCEloss = nn.BCEWithLogitsLoss()
 
                    for epoch in range(num_epochs):
                        epoch_loss = []
                        model.train()
                        for i in range(batch_per_e):

                            optimizer.zero_grad()
                            x,y, y1 = get_batch(xt, yt, y1t, 6)
                            out = model(x, y1) # get output from final linear layer

                            loss = SigBCEloss(out, y) # put through sigmoid and calculate loss
                            loss.backward()
                            optimizer.step()
                        
                        model.eval()

                        out = model(xv, y1v)             

                        yn = yv.cpu().detach().numpy()

                        ROC = roc_auc_score(yn, model.final(out).detach())
                        out = out.cpu().squeeze().detach().numpy()


                        preds = np.zeros(len(out))
                        pos = out.argsort()[-((yv==1).sum()) : ]
                        preds[pos] = 1

                        acc = accuracy_score(yn, preds)
                        f1 = f1_score(yn, preds)
                
                        if ROC > best_auc:
                            print("best UAC in cluster {}: {}".format(clust, ROC))
                            best_auc = ROC
                            best_f1 = f1
                            param_clust_hist[clust] = [lr, size, epoch]
                            clust_models[clust] = deepcopy(model.state_dict())

            # histories for this clust
            auc_clust_hist[clust] = best_auc
            f1_clust_hist[clust] = best_f1

        predictions = np.zeros(len(Y_val))
        for clust in train_l[0].unique():
        
            idxs = val_l[val_l[0] == clust].index
            xv = X_val[idxs]
            y1v = Y1_val[idxs]
            size = param_clust_hist[clust][1]
            if size == "0":
                model = Model()
                model.load_state_dict(clust_models[clust])
            elif size == "1":
                model = Model1()
                model.load_state_dict(clust_models[clust])
            elif size == "2":
                model = Model2()
                model.load_state_dict(clust_models[clust])
            else:
                size = Model3()
                model.load_state_dict(clust_models[clust])

            preds = model(xv, y1v)

            preds = preds.cpu().squeeze().detach().numpy()
            predictions[idxs] = preds

        pos_idxs = predictions.argsort()[- (Y_val == 1).sum() : ]
        Yhat = np.zeros(len(Y_val))
        Yhat[pos_idxs] = 1
        auc = roc_auc_score(Y_val, predictions)
        f1 = f1_score(Y_val, Yhat)

        if auc > tot_auc:
            print("auc!! {}".format(auc))
            tot_auc = auc
            tot_model = clust_models
            tot_param = param_clust_hist

    return tot_model, tot_param, tot_auc


In [127]:
n_epochs = 800
a = train_model(models, X_train, Y_train, Y_1_train, train_l, X_val, Y_val, Y_1_val,val_l, n_epochs, 0.00000051)

3: train: Counter({0: 4894, 1: 2882, 2: 1242}), val: Counter({0: 4897, 1: 2932, 2: 1282})
best UAC in cluster 2: 0.44609266276445475
best UAC in cluster 2: 0.4464210323313718
best UAC in cluster 2: 0.44648357891554646
best UAC in cluster 2: 0.4476198418613863
best UAC in cluster 2: 0.4485476161933107
best UAC in cluster 2: 0.44901150335927276
best UAC in cluster 2: 0.44959005926288853
best UAC in cluster 2: 0.4496864852468245
best UAC in cluster 2: 0.45054128856387826
best UAC in cluster 2: 0.4506403206554882
best UAC in cluster 2: 0.45066377562455373
best UAC in cluster 2: 0.45068462448594526
best UAC in cluster 2: 0.45099214519147074
best UAC in cluster 2: 0.4513309391890836
best UAC in cluster 2: 0.45233168453587835
best UAC in cluster 2: 0.4528581182860151
best UAC in cluster 2: 0.4538432269867662
best UAC in cluster 2: 0.45448954168990446
best UAC in cluster 2: 0.4547397280266031
best UAC in cluster 2: 0.4548335479028651
best UAC in cluster 2: 0.45514367471606454
best UAC in clust

best UAC in cluster 2: 0.5513585639304274
best UAC in cluster 2: 0.553021260626404
best UAC in cluster 2: 0.5531880515175366
best UAC in cluster 2: 0.555283362087388
best UAC in cluster 2: 0.5595156809498741
best UAC in cluster 2: 0.5610741333388931
best UAC in cluster 2: 0.5629036209260022
best UAC in cluster 2: 0.5630756240324825
best UAC in cluster 2: 0.5639095784881448
best UAC in cluster 2: 0.5646132275601099
best UAC in cluster 0: 0.4613992311196505
best UAC in cluster 0: 0.4615730724900304
best UAC in cluster 0: 0.5562789373739615
best UAC in cluster 0: 0.5566355853425249
best UAC in cluster 0: 0.5571453425921724
best UAC in cluster 0: 0.557637029304528
best UAC in cluster 0: 0.5581752231361151
best UAC in cluster 0: 0.5582921913426157
best UAC in cluster 0: 0.5584749979407992
best UAC in cluster 0: 0.5585902851670866
best UAC in cluster 0: 0.5587567022081916
best UAC in cluster 0: 0.5595725379383226
best UAC in cluster 0: 0.5597998905121555
best UAC in cluster 0: 0.560835934650

best UAC in cluster 0: 0.5860257033081131
best UAC in cluster 0: 0.5860420928651915
best UAC in cluster 0: 0.5860469957241466
best UAC in cluster 0: 0.5860691286302868
best UAC in cluster 0: 0.5860979854572799
best UAC in cluster 0: 0.5862778503400903
best UAC in cluster 0: 0.5863095088007719
best UAC in cluster 0: 0.5863340230955475
best UAC in cluster 0: 0.5864260567622193
best UAC in cluster 0: 0.5864514115471013
best UAC in cluster 0: 0.5864861518048404
best UAC in cluster 0: 0.5866057815633452
best UAC in cluster 0: 0.5866333776551783
best UAC in cluster 0: 0.5866522886825766
best UAC in cluster 0: 0.5869613088784331
best UAC in cluster 0: 0.5869854029281554
best UAC in cluster 0: 0.5870258865349564
best UAC in cluster 0: 0.5871026512980249
best UAC in cluster 0: 0.5871047525232913
best UAC in cluster 0: 0.587198046925123
best UAC in cluster 0: 0.5873426112234567
best UAC in cluster 0: 0.5875069270392952
best UAC in cluster 0: 0.5875822909855195
best UAC in cluster 0: 0.5876652193

best UAC in cluster 0: 0.597241623535516
best UAC in cluster 0: 0.5972612349713364
best UAC in cluster 0: 0.5972748228947264
best UAC in cluster 0: 0.5972892513082229
best UAC in cluster 0: 0.5973237114025931
best UAC in cluster 0: 0.5974296131560235
best UAC in cluster 0: 0.5974733186415663
best UAC in cluster 0: 0.5974926499140178
best UAC in cluster 0: 0.5975807612935256
best UAC in cluster 0: 0.5976066764051453
best UAC in cluster 0: 0.5976148011428424
best UAC in cluster 0: 0.5976433778064665
best UAC in cluster 0: 0.5977244851017525
best UAC in cluster 0: 0.5978635862143933
best UAC in cluster 0: 0.5978836178952671
best UAC in cluster 0: 0.5979500166136877
best UAC in cluster 0: 0.5980413498719372
best UAC in cluster 0: 0.5980629224513399
best UAC in cluster 0: 0.5980888375629597
best UAC in cluster 0: 0.5982046851159848
best UAC in cluster 0: 0.5982656206487127
best UAC in cluster 0: 0.5982835511043199
best UAC in cluster 0: 0.598287753554853
best UAC in cluster 0: 0.59841284649

best UAC in cluster 1: 0.4626792668887474
best UAC in cluster 1: 0.4634075864645079
best UAC in cluster 1: 0.4635966510312056
best UAC in cluster 1: 0.4640486454710233
best UAC in cluster 1: 0.464231969210797
best UAC in cluster 1: 0.46484049686474305
best UAC in cluster 1: 0.46540003612893743
best UAC in cluster 1: 0.46571042350462943
best UAC in cluster 1: 0.4658332772008035
best UAC in cluster 1: 0.46618117131239895
best UAC in cluster 1: 0.4667648220496742
best UAC in cluster 1: 0.467854430999853
best UAC in cluster 1: 0.4689773367461912
best UAC in cluster 1: 0.4698598932053103
best UAC in cluster 1: 0.4704779889041298
best UAC in cluster 1: 0.47090510642727673
best UAC in cluster 1: 0.4711741598491157
best UAC in cluster 1: 0.47144665776710915
best UAC in cluster 1: 0.4716908342722774
best UAC in cluster 1: 0.4722415709351884
best UAC in cluster 1: 0.47290903774555426
best UAC in cluster 1: 0.4734861822123157
best UAC in cluster 1: 0.47459110003429195
best UAC in cluster 1: 0.474

best UAC in cluster 1: 0.5521045106059863
best UAC in cluster 1: 0.5522974023906335
best UAC in cluster 1: 0.5524489602214275
best UAC in cluster 1: 0.5533158250869544
best UAC in cluster 1: 0.5535136922549355
best UAC in cluster 1: 0.5537548069857444
best UAC in cluster 1: 0.5538627345319159
best UAC in cluster 1: 0.5544919291627884
best UAC in cluster 1: 0.5547510318179592
best UAC in cluster 1: 0.554886132611571
best UAC in cluster 1: 0.5552416811590652
best UAC in cluster 1: 0.5555183890168031
best UAC in cluster 1: 0.5556282301719493
best UAC in cluster 1: 0.5560017666438055
best UAC in cluster 1: 0.55647213172978
best UAC in cluster 1: 0.5569042246362611
best UAC in cluster 1: 0.557139215818351
best UAC in cluster 1: 0.557221118282467
best UAC in cluster 1: 0.5575961856415029
best UAC in cluster 1: 0.5578307941017978
best UAC in cluster 1: 0.5583172335031598
best UAC in cluster 1: 0.5583451721941899
best UAC in cluster 1: 0.5585464838583256
best UAC in cluster 1: 0.55881362367118

best UAC in cluster 1: 0.5855321976191643
best UAC in cluster 1: 0.5856803109538039
best UAC in cluster 1: 0.5856871999461128
best UAC in cluster 1: 0.5858314860628031
best UAC in cluster 1: 0.5858927215499926
best UAC in cluster 1: 0.5861047494243865
best UAC in cluster 1: 0.5861809110615784
best UAC in cluster 1: 0.586207318865429
best UAC in cluster 1: 0.5862976412090334
best UAC in cluster 1: 0.5864668042423946
best UAC in cluster 1: 0.5866267819526771
best UAC in cluster 1: 0.5866895483270465
best UAC in cluster 1: 0.5867844633321903
best UAC in cluster 1: 0.5869750587860677
best UAC in cluster 1: 0.5871143695194239
best UAC in cluster 1: 0.5871905311566159
best UAC in cluster 1: 0.5872234452309804
best UAC in cluster 1: 0.5873750030617744
best UAC in cluster 1: 0.587441213932298
best UAC in cluster 1: 0.5875146965169256
best UAC in cluster 1: 0.5876214758977123
best UAC in cluster 1: 0.5877496876990154
best UAC in cluster 1: 0.5878595288541616
best UAC in cluster 1: 0.58796860456

best UAC in cluster 1: 0.5988922500367413
best UAC in cluster 1: 0.5989358803213638
best UAC in cluster 1: 0.5989917577034243
best UAC in cluster 1: 0.5990683020624112
best UAC in cluster 1: 0.5991383401508842
best UAC in cluster 1: 0.5992152672316662
best UAC in cluster 1: 0.5992466504188507
best UAC in cluster 1: 0.599304058688091
best UAC in cluster 1: 0.599408924459903
best UAC in cluster 1: 0.5995160865624847
best UAC in cluster 1: 0.599590334590702
best UAC in cluster 1: 0.5996393229804536
best UAC in cluster 1: 0.5997143364522608
best UAC in cluster 1: 0.5998211158330476
best UAC in cluster 1: 0.5998406346445893
best UAC in cluster 1: 0.5998590052907461
best UAC in cluster 1: 0.5998762277715182
best UAC in cluster 1: 0.5998873267035713
best UAC in cluster 1: 0.5999271297702444
best UAC in cluster 1: 0.5999684637240974
best UAC in cluster 1: 0.6000090322343605
best UAC in cluster 1: 0.6000476871356489
best UAC in cluster 1: 0.6000851938715523
best UAC in cluster 1: 0.600112749840

In [128]:
a[1]

{2: [5.1e-06, '2', 93],
 0: [5.1e-06, Model3(
    (net): Sequential(
      (0): Conv2d(39, 5, kernel_size=(4, 4), stride=(1, 1))
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ReLU()
      (3): Dropout2d(p=0.3, inplace=False)
      (4): Conv2d(5, 5, kernel_size=(4, 4), stride=(1, 1))
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): ReLU()
      (7): Dropout2d(p=0.3, inplace=False)
      (8): Flatten(start_dim=1, end_dim=-1)
      (9): Linear(in_features=45, out_features=1, bias=True)
    )
    (final): Sigmoid()
  ), 242],
 1: [5.1e-06, '1', 289]}

In [4]:
predictions = np.zeros(len(Y_test))
auc_clusters = {}
f1_clusters = {}
for clust in train_l[0].unique():
  
    idxs = test_l[test_l[0] == clust].index
    xt = torch.tensor(X_test[idxs]).float().to(device)
    yt = Y_test[idxs]
    y1 = Y_1_test[idxs]

    size = a[1][clust][1]
    if size == "0":
        model = Model()
        model.load_state_dict(a[0][clust])
        model.eval()
    elif size == "1":
        model = Model1()
        model.load_state_dict(a[0][clust])
        model.eval()
    elif size == "2":
        model = Model2()
        model.load_state_dict(a[0][clust])
        model.eval()
    else:
        size = Model3()
        model.load_state_dict(a[0][clust])
        model.eval()

    preds = model(xt, y1)
    preds = preds.cpu().squeeze().detach().numpy()  
    auc = roc_auc_score(yt, preds)
    auc_clusters[clust] = auc

    Yhat = np.zeros(len(yt))
    pos_idxs = preds.argsort()[- (yt == 1).sum() : ]
    Yhat[pos_idxs] = 1
    f1_clusters[clust] = f1 = f1_score(yt, Yhat)

    predictions[idxs] = preds

pos_idxs = predictions.argsort()[- (Y_test == 1).sum() : ]
Yhat = np.zeros(len(Y_test))
Yhat[pos_idxs] = 1
auc = roc_auc_score(Y_test, predictions)
f1 = f1_score(Y_test, Yhat)
print(auc,f1,f1_clusters, auc_clusters)

name = "Denhaag"

# with open ("../Results/" + name + ".csv", "a+") as f:
#     f.write("\nCNN_cluster,{},{}".format(auc,f1))
    
# with open ("../Results/" + name + "AUC.csv", "a+") as f:
#     f.write("\nCNN_clust,{},{},{}".format(auc_clusters[0],auc_clusters[1],auc_clusters[2]))
    
# with open ("../Results/" + name + "f1.csv", "a+") as f:
#     f.write("\nCNN_clust,{},{},{}".format(f1_clusters[0],f1_clusters[1],f1_clusters[2]))
    
# with open ("../Results/modellen.csv", "a") as f:
#     f.write("\n" + name + ",CNNcluster," + str(a[1]))

NameError: name 'np' is not defined

In [3]:
import pickle
f = open('../results/NMCNNpreds','wb')
pickle.dump(out,f)
f.close()

NameError: name 'out' is not defined

In [ ]:
print(np.max(a[-2]))
plt.plot(a[-2], alpha = 0.5)
plt.hlines(np.mean(a[-2]), 0, n_epochs, color = "r")

In [37]:
name = "Eindhoven"

BEST CNN FOUND Den haaG: AUC 0.636, model: 2, lr = 5.1e-06


In [ ]:
def neighbor_part(model, X, idxs, w):
    preds = model(X)
    preds = pd.DataFrame(np.array(preds.detach())).set_index(idxs)
    
    neighbors = [w.neighbors[x] for x in idxs]
    transitions = [preds.loc[x].values for x in neighbors]



    n_function = np.zeros((len(preds), w.max_neighbors + 1))
    for i, (t, idx) in enumerate(zip(transitions, idxs)):
        n_function[i, 1:len(t) + 1] = t.squeeze()
        n_function[i, 0] = preds.loc[idx]
        
    return n_function

In [ ]:
class Model(nn.Module): # best auc = 0.59
    def __init__(self, name):
        super(Model, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 60, kernel_size = (3,3)),
            nn.MaxPool2d(2),
            nn.ReLU(), 
            nn.Dropout2d(0.25),
            nn.Conv2d(in_channels = 60, out_channels = 60, kernel_size = (3,3)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout2d(0.25),
            nn.Flatten(),
            nn.Linear(60, 1))
        
        self.final = nn.Sigmoid()
        self.name = name
        
    def forward(self, x, y1):
#         for i in self.net:
            
        out = self.net(x)
        out = out.squeeze() * y1
        return out

# Neighbor function

In [ ]:
import geopandas as gpd
import shapely
import libpysal

def load_data(year):
    os.getcwd()
    df = pd.DataFrame()
    path = "../Data/filled/" + str(year) + "/"
    for filename in os.listdir(path):
        df1 = pd.read_csv(path + filename)
        if df1.geometry.isna().any():
            print(filename)
        df = pd.concat([df, df1])
    
    df = gpd.GeoDataFrame(df)
    df.geometry = df.geometry.apply(shapely.wkt.loads)
    
    df = df.reset_index()
    df = df.drop(["Unnamed: 0", "index"], axis = 1)
    return df

In [ ]:
df15 = load_data(2015)
df16 = load_data(2016)
wt = libpysal.weights.DistanceBand.from_dataframe(df15, threshold=150, binary = True, silence_warnings = True)
wv = libpysal.weights.DistanceBand.from_dataframe(df16, threshold=150, binary = True, silence_warnings = True)

In [ ]:

scaler = StandardScaler()
X15 = df15[df15.columns[:-4]]
Y15 = df15.y

X16 = df16[df16.columns[:-4]]
Y16 = df16.y

X15 = pd.DataFrame(scaler.fit_transform(X15))
X16 = pd.DataFrame(scaler.transform(X16))

In [ ]:
cluster_model = KMeans(n_clusters=5, random_state=0)
labels15 = pd.DataFrame(cluster_model.fit_predict(X15))
labels16 = pd.DataFrame(cluster_model.predict(X16))

labels15["X"] = X15.index
labels16["X"] = X16.index


labels15 = labels15.set_index(0)
labels16 = labels16.set_index(0)

In [ ]:
with open('cbs_id_koppel.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
# br.C28992R100 = br.C28992R100.map(b) # change C28992code for id
labels15.C28 = labels15.C28.map(b)

In [ ]:
labels15

In [ ]:
X_train[labels15.loc[0]["X"]].shape

In [ ]:
labels15.loc[0].shape

In [ ]:
ID_train

In [ ]:

# preds = pd.DataFrame(np.array(preds.detach())).set_index(ID_train)
neighbors = [w15.neighbors[x] for x in ID_train]
transitions = [preds.loc[x].values for x in neighbors]



n_function = np.zeros((len(preds), w15.max_neighbors + 1))
for i, (t, idx) in enumerate(zip(transitions, ID_train)):
    n_function[i, 1:len(t) + 1] = t.squeeze()
    n_function[i, 0] = preds.loc[idx]

In [ ]:
params = {"max_depth": [6, 8, 10, 12], "oob_score" : [True, False]}
clf_bagger = GridSearchCV(RandomForestClassifier(), params, cv = 5, scoring = "balanced_accuracy",
                               verbose = 3)
oversample = SMOTE()
x, y = oversample.fit_resample(n_function, Y_train)

clf_bagger.fit(x,y)

In [ ]:
clf_bagger.best_params_

In [ ]:
np.array(preds.detach())

In [ ]:
X_val = torch.from_numpy(X_val).float()
preds = model(X_val)

In [ ]:
with open('w16.pickle', 'rb') as handle:
    w16 = pickle.load(handle)

In [ ]:
preds = pd.DataFrame(np.array(preds.detach())).set_index(ID_val)
neighbors = [w16.neighbors[x] for x in ID_val]
transitions = [preds.loc[x].values for x in neighbors]



n_function = np.zeros((len(preds), w16.max_neighbors + 1))
for i, (t, idx) in enumerate(zip(transitions, ID_val)):
    n_function[i, 1:len(t) + 1] = t.squeeze()
    n_function[i, 0] = preds.loc[idx]

In [ ]:
preds = clf_bagger.predict(n_function)

In [ ]:
len(neighbors)

In [ ]:
clf_bagger.score(n_function, Y_val)

In [ ]:
recall_score(Y_val, preds)

In [ ]:
precision_score(Y_val, preds)

In [ ]:
(Y_val == 1).sum()

# No bagger on the end

In [ ]:
preds = pd.DataFrame(np.array(preds.detach())).set_index(ID_val)

In [ ]:
preds["y"] = Y_val

In [ ]:
preds = preds.sort_values(by=[0], ascending = False)

In [ ]:
preds

In [ ]:
preds[0].iloc[0:4529] = 1
preds[0].iloc[4529:] = 0

In [ ]:
recall_score(preds["y"], preds[0])


In [ ]:
precision_score(preds["y"], preds[0])